In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,9,11,12,13,16,...,48,49,54,55,56,57,59,60,62,63
2,33.96,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,...,0.00,-0.00,0.0,0.00,-0.00,-0.00,0.0,0.00,0.0,-0.00
3,0.00,9.09,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,...,0.00,-0.00,-0.0,-0.00,0.00,0.00,-0.0,0.00,-0.0,-0.00
4,-0.00,-0.00,38.58,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,...,-0.00,-0.00,-0.0,-0.00,0.00,0.00,0.0,0.00,0.0,0.00
5,-0.00,0.00,-0.00,27.82,0.00,-0.00,0.00,0.00,0.00,-0.00,...,-0.00,0.00,0.0,-0.00,0.00,0.00,-0.0,-0.00,-0.0,0.00
6,0.00,-0.00,-0.00,0.00,15.44,-0.00,-0.00,0.00,0.00,-0.00,...,-0.00,0.00,0.0,-0.00,-0.00,-0.00,-0.0,0.00,0.0,-0.00
9,0.00,-0.00,0.00,-0.00,-0.00,4.28,-0.00,-0.00,0.00,0.00,...,-0.00,-0.00,0.0,-0.00,-0.00,-0.00,-0.0,0.00,0.0,-0.00
11,0.00,0.00,0.00,0.00,-0.00,-0.00,14.25,0.00,0.00,0.00,...,0.00,0.00,0.0,-0.00,-0.00,-0.00,0.0,-0.00,0.0,0.00
12,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,9.29,-0.00,-0.00,...,-0.00,-0.00,0.0,-0.00,-0.00,0.00,0.0,-0.00,-0.0,0.00
13,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,11.05,-0.00,...,0.00,-0.00,0.0,0.00,-0.00,0.00,-0.0,0.00,-0.0,-0.00
16,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,8.13,...,-0.00,0.00,-0.0,0.00,0.00,-0.00,0.0,0.00,-0.0,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,9,11,12,13,16,...,48,49,54,55,56,57,59,60,62,63
2,1.00000,0.00002,-0.00001,-0.00002,0.00002,0.00001,0.00004,0.00000,-0.00002,-0.00002,...,0.00001,-0.00002,0.00004,0.00000,-0.00002,-0.00004,0.00003,0.00000,0.00004,-0.00002
3,0.00002,1.00000,-0.00001,0.00002,-0.00002,-0.00000,0.00004,-0.00001,-0.00000,0.00005,...,0.00001,-0.00004,-0.00001,-0.00002,0.00000,0.00002,-0.00003,0.00004,-0.00004,-0.00001
4,-0.00001,-0.00001,1.00000,-0.00005,-0.00001,0.00002,0.00003,-0.00010,-0.00001,0.00000,...,-0.00002,-0.00002,-0.00006,-0.00002,0.00004,0.00005,0.00001,0.00007,0.00003,0.00002
5,-0.00002,0.00002,-0.00005,1.00000,0.00001,-0.00004,0.00005,0.00006,0.00003,-0.00004,...,-0.00001,0.00000,0.00000,-0.00004,0.00001,0.00002,-0.00001,-0.00005,-0.00001,0.00004
6,0.00002,-0.00002,-0.00001,0.00001,1.00000,-0.00003,-0.00000,0.00000,0.00002,-0.00001,...,-0.00006,0.00003,0.00002,-0.00006,-0.00006,-0.00004,-0.00001,0.00003,0.00003,-0.00006
9,0.00001,-0.00000,0.00002,-0.00004,-0.00003,1.00000,-0.00003,-0.00004,0.00006,0.00001,...,-0.00001,-0.00001,0.00008,-0.00002,-0.00004,-0.00001,-0.00004,0.00003,0.00001,-0.00003
11,0.00004,0.00004,0.00003,0.00005,-0.00000,-0.00003,1.00000,0.00008,0.00002,0.00005,...,0.00000,0.00004,0.00000,-0.00001,-0.00000,-0.00003,0.00002,-0.00005,0.00004,0.00006
12,0.00000,-0.00001,-0.00010,0.00006,0.00000,-0.00004,0.00008,1.00000,-0.00004,-0.00004,...,-0.00002,-0.00002,0.00005,-0.00003,-0.00003,0.00001,0.00002,-0.00000,-0.00004,0.00003
13,-0.00002,-0.00000,-0.00001,0.00003,0.00002,0.00006,0.00002,-0.00004,1.00000,-0.00001,...,0.00002,-0.00002,0.00005,0.00001,-0.00002,0.00004,-0.00004,0.00002,-0.00002,-0.00001
16,-0.00002,0.00005,0.00000,-0.00004,-0.00001,0.00001,0.00005,-0.00004,-0.00001,1.00000,...,-0.00001,0.00003,-0.00003,0.00005,0.00002,-0.00001,0.00001,0.00003,-0.00002,0.00003


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.016715359829112333

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.70710867e+01 3.78947353e+01 3.61037706e+01 3.11538560e+01
 2.78452714e+01 2.65716551e+01 2.40967555e+01 2.08926082e+01
 1.86626431e+01 1.77831448e+01 1.63416969e+01 1.58387310e+01
 1.31127417e+01 7.41851229e+00 6.23381434e+00 4.66349626e+00
 3.83076031e+00 3.43873061e+00 3.12770906e+00 2.72963321e+00
 2.45661690e+00 2.09486617e+00 1.86788184e+00 1.59061270e+00
 1.48528101e+00 1.12337277e+00 8.16889809e-01 7.21404663e-01
 6.29513039e-01 5.02290277e-01 1.14656066e-01 1.52777753e-30]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.082846,-0.004938,0.987604,-0.030468,-0.009443,-0.002214,-0.008450,-0.005109,-0.006172,-0.004353,...,-0.003602,-0.020559,-0.001989,-0.005796,-0.035806,-0.006941,-0.037319,-0.000624,-0.017070,-0.002480
1,-0.472877,-0.002503,0.037207,-0.021034,-0.005037,-0.001103,-0.004475,-0.002558,-0.003184,-0.002204,...,-0.001832,-0.012368,-0.001008,-0.002954,-0.027087,-0.003566,-0.029265,-0.000312,-0.010023,-0.001272
2,0.855004,-0.007425,0.088808,-0.068660,-0.014893,-0.003289,-0.013200,-0.007624,-0.009438,-0.006567,...,-0.005383,-0.038248,-0.002921,-0.008783,-0.093075,-0.010755,-0.100228,-0.000940,-0.030085,-0.003727
3,0.110734,-0.006310,0.045481,-0.088892,-0.012998,-0.002751,-0.011468,-0.006502,-0.008018,-0.005540,...,-0.004557,-0.038480,-0.002448,-0.007504,-0.158074,-0.009230,-0.189813,-0.000777,-0.028693,-0.003146
4,0.022681,-0.002306,0.012125,-0.061395,-0.004800,-0.000995,-0.004212,-0.002317,-0.002946,-0.001996,...,-0.001648,-0.016696,-0.000879,-0.002686,-0.577297,-0.003364,0.810899,-0.000287,-0.011681,-0.001099
5,0.075690,-0.009027,0.042462,-0.411636,-0.019469,-0.003914,-0.016896,-0.009282,-0.011625,-0.007856,...,-0.006450,-0.073432,-0.003454,-0.010703,0.736715,-0.013377,0.470460,-0.001108,-0.049380,-0.004378
6,0.070962,-0.011585,0.044209,0.871497,-0.025768,-0.005000,-0.022154,-0.011870,-0.014980,-0.010142,...,-0.008246,-0.123958,-0.004407,-0.013913,0.233842,-0.017380,0.203857,-0.001424,-0.074095,-0.005579
7,0.041205,-0.009846,0.028280,0.135249,-0.023169,-0.004214,-0.019803,-0.010156,-0.012959,-0.008614,...,-0.007004,-0.224881,-0.003691,-0.011960,0.087257,-0.015143,0.081373,-0.001170,-0.088235,-0.004730
8,0.006026,-0.001887,0.004381,0.014468,-0.004577,-0.000798,-0.003876,-0.001957,-0.002530,-0.001603,...,-0.001307,0.762541,-0.000686,-0.002232,0.011167,-0.002934,0.010591,-0.000226,-0.026334,-0.000891
9,0.036395,-0.012495,0.026528,0.080454,-0.031700,-0.005169,-0.026273,-0.012837,-0.016548,-0.010784,...,-0.008646,0.506902,-0.004572,-0.015266,0.062100,-0.019693,0.059304,-0.001466,-0.244279,-0.005837


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([1.08284587, 1.00250346, 0.91119246, 1.0888918 , 1.00480039,
       1.00391362, 1.02215374, 1.01015551, 1.00253016, 1.01078383,
       0.19672488, 0.9992052 , 0.91605156, 1.02501993, 1.01762391,
       1.01863462, 1.02627504, 0.99000961, 0.22582425, 0.99541257,
       0.97712201, 0.99473376, 1.25962139, 0.99088488, 0.99621481,
       1.09807468, 0.99200146, 1.00801194, 1.00751907, 0.97770087,
       1.04636181, 0.79891273])